# create_metadata_table

This notebook produces a table of PMC paper IDs matched to the SRA or GEO datasets they contain, and metadata about those datasets.

In [1]:
# import required modules
import numpy as np
import pandas as pd
from collections import Counter
import statistics

### Import and clean the data

In [2]:
# import GEO reference data

# import table with accession, platform, and series
geoAPS = pd.read_csv('../geo_sample.csv')

# import table with datasets IDs
geoSeries = pd.read_csv('../geo_series_table.csv', low_memory = False)
geoSeries.rename(columns={'Accession':'Series'}, inplace = True)

# add datasets column by merging
allData = pd.merge(geoAPS, geoSeries, how = 'outer', on = 'Series')
geoReference = allData[['Series', 'Accession', 'Platform', 'Datasets']].drop_duplicates()

# geoReference

In [3]:
# import SRA reference data
sraReference = pd.read_csv('../sraIDfull.csv', error_bad_lines = False, low_memory=False, quoting=3)
sraReference = sraReference[['SRAStudy', 'Run', 'Experiment', 
                             'BioProject', 'Submission', 'Sample']].drop_duplicates()
# sraReference

In [4]:
# import data scraped from PubMed
# this data was scraped from XML files on the hoffman2 cluster: /u/scratch/n/nikodm/pmcOA/

pmcData = pd.read_csv('../data_tables/preFilterMatrix.csv').drop_duplicates()
# pmcData

In [5]:
# Merge GEO accessions with reference data, convert to Series

# match series first
se = pd.merge(pmcData, geoReference['Series'].drop_duplicates(), how = 'left',
            left_on = 'accession', right_on = 'Series')
pmcData = se.rename(columns = {'Series': 'Series_result'})

# match each other style of GEO ID
for col in ['Accession', 'Platform', 'Datasets']:
    pmcData = pd.merge(pmcData, geoReference[['Series', col]].drop_duplicates(subset = col), how = 'left', 
            left_on = 'accession', right_on = col)
    label = col + '_result'
    pmcData = pmcData.rename(columns = {'Series': label})

# combine all GEO series match columns into one aggregate GEO series column, clean up
pmcData['geoSeries'] = pmcData['Series_result'].fillna(pmcData['Accession_result']).fillna(pmcData['Platform_result']).fillna(pmcData['Datasets_result'])
pmcData = pmcData.drop(labels = ['Accession', 'Platform', 'Datasets',
                           'Series_result', 'Accession_result',
                           'Platform_result', 'Datasets_result'], axis = 1)
pmcData_geoMerged = pmcData
# pmcData_geoMerged

In [6]:
# merge SRA accessions with reference data, convert to Study

# match SRA Study IDs first
st = pd.merge(pmcData_geoMerged, sraReference['SRAStudy'].drop_duplicates(), how = 'left',
            left_on = 'accession', right_on = 'SRAStudy')
pmcData = st.rename(columns = {'SRAStudy': 'Study_result'})

# match every other style of SRA ID
for col in ['Run', 'Experiment', 'BioProject', 'Submission', 'Sample']:
    pmcData = pd.merge(pmcData, sraReference[['SRAStudy', col]].drop_duplicates(subset = [col]), how = 'left',
                      left_on = 'accession', right_on = col)
    label = col + '_result'
    pmcData = pmcData.rename(columns = {'SRAStudy': label})

# combine all SRA Study matches into one aggregate column, clean up
pmcData['sraStudy'] = pmcData['Study_result'].fillna(pmcData['Run_result']).fillna(pmcData['Experiment_result']).fillna(pmcData['BioProject_result']).fillna(pmcData['Submission_result']).fillna(pmcData['Sample_result'])
pmcData = pmcData.drop(labels = ['Run', 'Experiment', 'BioProject', 'Submission', 'Sample',
                                'Study_result', 'Run_result', 'Experiment_result', 'BioProject_result',
                                'Submission_result', 'Sample_result'], axis = 1)

pmcData_sraMerged = pmcData
# pmcData_sraMerged

In [7]:
# combine GEO Series hits and SRA study hits into one converted accession column
pmcData['converted_accession'] = pmcData_sraMerged['geoSeries'].fillna(pmcData_sraMerged['sraStudy'])
pmcData = pmcData.drop(labels = ['geoSeries', 'sraStudy'], axis = 1)
# pmcData

In [8]:
# clean out garbage converted_accession entries, and rows that didn't map to a converted_accession
w = []
for a in pmcData['converted_accession']:
    if(type(a) == str):
        if a[0:3] != 'GSE' and a[0:3] != 'GPL' and a[1:3] != 'RP' and a not in w:
                w.append(a)
w.append(np.NaN)
        
pmcData = pmcData[~pmcData.converted_accession.isin(w)]
pmcData

,journal,pmc_ID,accession,converted_accession
0,Alzheimers_Res_Ther,PMC3707052,GSM1,GSE506
1,Alzheimers_Res_Ther,PMC3706879,GSE45534,GSE45534
2,Alzheimers_Res_Ther,PMC4255636,GSE5281,GSE5281
3,Alzheimers_Res_Ther,PMC4731966,GSE67036,GSE67036
4,Alzheimers_Res_Ther,PMC4731966,GSE1297,GSE1297
...,...,...,...,...
243773,Saudi_J_Biol_Sci,PMC6088103,GSE28146,GSE28146
243775,Saudi_J_Biol_Sci,PMC6933160,ERS2923991,ERP109196
243776,Saudi_J_Biol_Sci,PMC6933160,ERS2923992,ERP109196
243777,Saudi_J_Biol_Sci,PMC6933160,ERS2923995,ERP109196


In [9]:
# perform QC with gold standard from Penn group (Casey + Kurt)
gsPMC = pd.read_table("../pubmed_mappings.tsv")
gsPMC.columns = ["SRA_accession_code", "GEO_accession_code", "pm_ID", "pmc_ID"]
gsPMC

,SRA_accession_code,GEO_accession_code,pm_ID,pmc_ID
0,SRP111833,GSE101341,29535194.0,PMC5850328
1,ERP108370,NaN,NaN,NaN
2,SRP062170,GSE71840,27780967.0,NaN
3,ERP114122,NaN,NaN,NaN
4,SRP162020,GSE120109,30692590.0,PMC6349857
...,...,...,...,...
26191,SRP075828,GSE82005,27373156.0,PMC4956617
26192,SRP045359,GSE60226,25722416.0,PMC4428148
26193,SRP123344,GSE106413,NaN,NaN
26194,SRP199106,GSE131584,32421711.0,PMC7259794


In [10]:
# QC step: de-duplicate datasets present in both SRA and GEO
gsPMC_acc = gsPMC[['SRA_accession_code', 'GEO_accession_code']].dropna().drop_duplicates()
gsPMC_acc

,SRA_accession_code,GEO_accession_code
0,SRP111833,GSE101341
2,SRP062170,GSE71840
4,SRP162020,GSE120109
5,SRP041755,GSE57401
6,SRP153370,GSE117074
...,...,...
26190,SRP124774,GSE106767
26191,SRP075828,GSE82005
26192,SRP045359,GSE60226
26193,SRP123344,GSE106413


In [11]:
ovAcc = pd.merge(pmcData, gsPMC_acc, how = "left", left_on = "converted_accession", right_on = "SRA_accession_code")
ovAcc

,journal,pmc_ID,accession,converted_accession,SRA_accession_code,GEO_accession_code
0,Alzheimers_Res_Ther,PMC3707052,GSM1,GSE506,NaN,NaN
1,Alzheimers_Res_Ther,PMC3706879,GSE45534,GSE45534,NaN,NaN
2,Alzheimers_Res_Ther,PMC4255636,GSE5281,GSE5281,NaN,NaN
3,Alzheimers_Res_Ther,PMC4731966,GSE67036,GSE67036,NaN,NaN
4,Alzheimers_Res_Ther,PMC4731966,GSE1297,GSE1297,NaN,NaN
...,...,...,...,...,...,...
223959,Saudi_J_Biol_Sci,PMC6088103,GSE28146,GSE28146,NaN,NaN
223960,Saudi_J_Biol_Sci,PMC6933160,ERS2923991,ERP109196,NaN,NaN
223961,Saudi_J_Biol_Sci,PMC6933160,ERS2923992,ERP109196,NaN,NaN
223962,Saudi_J_Biol_Sci,PMC6933160,ERS2923995,ERP109196,NaN,NaN


In [12]:
# count the number of SRA datasets also present in GEO
numDupSRA = len(ovAcc["SRA_accession_code"]) - ovAcc["SRA_accession_code"].isna().sum()
print("duplicated SRA datasets: " + str(numDupSRA))

duplicated SRA datasets: 3570


In [13]:
# convert SRA ID of duplicated datasets to GEO ID
ovAccNA = ovAcc.loc[ovAcc["SRA_accession_code"].isna(), :]
ovAcc = ovAcc.loc[~ovAcc["SRA_accession_code"].isna(), :]
ovAcc['converted_accession'] = ovAcc["GEO_accession_code"]
ovAccNA

/Users/niko/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,journal,pmc_ID,accession,converted_accession,SRA_accession_code,GEO_accession_code
0,Alzheimers_Res_Ther,PMC3707052,GSM1,GSE506,NaN,NaN
1,Alzheimers_Res_Ther,PMC3706879,GSE45534,GSE45534,NaN,NaN
2,Alzheimers_Res_Ther,PMC4255636,GSE5281,GSE5281,NaN,NaN
3,Alzheimers_Res_Ther,PMC4731966,GSE67036,GSE67036,NaN,NaN
4,Alzheimers_Res_Ther,PMC4731966,GSE1297,GSE1297,NaN,NaN
...,...,...,...,...,...,...
223959,Saudi_J_Biol_Sci,PMC6088103,GSE28146,GSE28146,NaN,NaN
223960,Saudi_J_Biol_Sci,PMC6933160,ERS2923991,ERP109196,NaN,NaN
223961,Saudi_J_Biol_Sci,PMC6933160,ERS2923992,ERP109196,NaN,NaN
223962,Saudi_J_Biol_Sci,PMC6933160,ERS2923995,ERP109196,NaN,NaN


In [14]:
pmcData = pd.concat([ovAcc, ovAccNA], axis = 0) 
pmcData = pmcData[['journal', 'pmc_ID', 'accession', 'converted_accession']]
pmcData

,journal,pmc_ID,accession,converted_accession
90,Biol_Direct,PMC4862193,SRR392998,GSE34743
91,Biol_Direct,PMC4862193,SRR393000,GSE34743
92,Biol_Direct,PMC4862193,SRR393006,GSE34743
93,Biol_Direct,PMC4862193,SRR393008,GSE34743
96,Biol_Direct,PMC4862193,SRR364364,GSE33671
...,...,...,...,...
223959,Saudi_J_Biol_Sci,PMC6088103,GSE28146,GSE28146
223960,Saudi_J_Biol_Sci,PMC6933160,ERS2923991,ERP109196
223961,Saudi_J_Biol_Sci,PMC6933160,ERS2923992,ERP109196
223962,Saudi_J_Biol_Sci,PMC6933160,ERS2923995,ERP109196


In [15]:
# # QC step: validate our publication-dataset relationships with gold standard table
# # merge my PMC ID-accession information with gold standard
# # in this table: only those papers in my data AND gold standard
# ovPMC = pd.merge(gsPMC[['pmc_ID', 'SRA_accession_code', 'GEO_accession_code']],
#                  pmcData[['pmc_ID', 'accession', 'converted_accession']], 
#                  on = "pmc_ID", how = "inner")
# # ovPMC

In [16]:
# # check for matches between both accession and converted accession, SRA and GEO
# ovPMC['match'] = (ovPMC['converted_accession'] == ovPMC['GEO_accession_code']).astype(int)
# ovPMC['match'] += (ovPMC['converted_accession'] == ovPMC['SRA_accession_code']).astype(int)
# ovPMC['match'] += (ovPMC['accession'] == ovPMC['GEO_accession_code']).astype(int)
# ovPMC['match'] += (ovPMC['accession'] == ovPMC['SRA_accession_code']).astype(int)
# ovPMC['match'] = ovPMC['match'] > 0
# # ovPMC

In [17]:
# # group by paper: if a paper had at least one successful match, we count it as a success
# ovPMC_byPap = ovPMC[['pmc_ID', 'match']].groupby("pmc_ID", as_index = False)['match'].max()
# ovPMC_byPap = pd.DataFrame(ovPMC_byPap)
# # ovPMC_byPap

# # ovPMC_byPap.loc[~ovPMC_byPap['match'], :]

In [18]:
# # count up results and report percentage
# totRelsSub = len(ovPMC_byPap['pmc_ID'])
# print("Number of papers overlapping with gold standard: " + str(totRelsSub))
# totMatching = ovPMC_byPap['match'].sum()
# print("Number of such papers validated by gold standard: " + str(totMatching))
# pctVal = totMatching / totRelsSub
# print("Percent validated: " + str(pctVal))

### Collect desired factors

In [19]:
# import SRA attribute data
# CAUTION: large file! Time delay on import...
sraAttributes = pd.read_csv('../sra_complete_runs.csv', error_bad_lines = False, low_memory=False)    

In [20]:
# Convert SRA dates to a universal format
pd.set_option('display.max_columns', 50)
sraAttributes['ReleaseDate'] = sraAttributes['ReleaseDate'].str[0:10]
# sraAttributes

In [21]:
# Define functions to convert GEO dates to a universal format
def strToMonth(m):
    if(m == 'Jan'):
        return '01'
    elif(m == 'Feb'):
        return '02'
    elif(m == 'Mar'):
        return '03'
    elif(m == 'Apr'):
        return '04'
    elif(m == 'May'):
        return '05'
    elif(m == 'Jun'):
        return '06'
    elif(m == 'Jul'):
        return '07'
    elif(m == 'Aug'):
        return '08'
    elif(m == 'Sep'):
        return '09'
    elif(m == 'Oct'):
        return '10'
    elif(m == 'Nov'):
        return '11'
    elif(m == 'Dec'):
        return '12'
    else:
        return(np.NaN)
    
def convGEODate(d):
    if(type(d) == str):
        mon = strToMonth(d[0:3])
        day = d[4:6]
        yr = d[8:12]
        return yr + '-' + mon + '-' + day
    else:
        return np.NaN

In [22]:
# import GEO attribute data and add Series column
geoPlatforms = pd.read_csv('../geo_platforms_table.csv')
geoPlatforms.rename(columns={'Accession':'Platform'}, inplace = True)
techByPlatform = geoPlatforms[['Platform', 'Technology']]

# allData contains metadata matched to GEO series, but lacks 'Technology' column
geoAttributes = pd.merge(allData, techByPlatform, how = 'left', on = 'Platform')
# geoAttributes

In [23]:
# convert GEO dates to universal format
dates = []
for i in geoAttributes['Release Date']:
    dates.append(convGEODate(i))
geoAttributes['Release Date'] = dates
# geoAttributes

In [24]:
# Add a column tagging each accession as GEO or SRA

repoList = []

for i in pmcData['converted_accession']:
    if(type(i) == str):
        if('GSE' in i or 'GPL' in i):
            repoList.append('GEO')
        elif('SRP' in i or 'ERP' in i or 'DRP' in i):
            repoList.append('SRA')
        else:
            repoList.append(np.NaN)
    else:
        repoList.append(np.NaN)
        
pmcData['repository'] = repoList
# pmcData

In [25]:
# add column for paper publish date
# this data was scraped from XML files on the hoffman2 cluster: /u/scratch/n/nikodm/pmcOA/

pmc_dates = pd.read_csv('../data_lists/preFilterDates.csv').drop_duplicates()

pmcData = pd.merge(pmcData, pmc_dates, how = 'left', on = 'pmc_ID')
pmcData = pmcData.rename(columns = {'date': 'pmc_date'})

# pmcData

In [26]:
# Get every factor we're interested in from our tables of GEO and SRA metadata...

# take a slice of the GEO and SRA attribute tables with only the info we want
slicedGEOAtt = geoAttributes[['Series', 'Release Date', 'Technology', 'Taxonomy']]
slicedGEOAtt.columns = ['converted_accession', 'geoRelease', 'geoHardware', 'geoSpecies']
slicedGEOAtt = slicedGEOAtt.drop_duplicates(subset = ['converted_accession'])

slicedSRAAtt = sraAttributes[['SRAStudy', 'ReleaseDate', 'Model', 
                              'LibraryStrategy', 'ScientificName', 
                              'bases', 'avgLength', 'Consent']]
slicedSRAAtt.columns = ['converted_accession', 'sraRelease', 'sraHardware', 
                        'sraLibrary_strategy', 'sraSpecies', 
                        'sraBases', 'sraAvg_length', 'sraAccess']
slicedSRAAtt = slicedSRAAtt.drop_duplicates(subset = ['converted_accession'])

In [27]:
# special case for GEO: make an educated guess on library strategy based on hardware
# These guesses are based on manually checking GEO series IDs that corresponded to various types of hardware

gc = Counter(geoAttributes['Technology'])
ls_guesses = pd.DataFrame.from_dict(gc, orient='index').reset_index()
ls_guesses.columns = ['hardware', 'use_count']
ls_guesses = ls_guesses.drop(labels = ['use_count'], axis = 1)

ls = []

for i in ls_guesses['hardware']:
    if(i == 'high-throughput sequencing'):
        ls.append('RNA-Seq')
    elif(i == 'SAGE NlaIII' or i == 'spotted DNA/cDNA' or i == 'SAGE Sau3A' 
         or i == 'in situ oligonucleotide' or i == 'spotted oligonucleotide' 
         or i == 'antibody' or i == 'MPSS' or i == 'oligonucleotide beads' 
         or i == 'RT-PCR' or i == 'mixed spotted oligonucleotide/cDNA' 
         or i == 'spotted peptide or protein'):
        ls.append('Expression_Array')
    else:
        ls.append(np.NaN)
        
ls_guesses.loc[:,'geoLibrary_strategy'] = ls
# ls_guesses

In [28]:
# merge SRA attributes onto pmcData table
mergedSRA = pd.merge(pmcData, slicedSRAAtt, how = 'left', on = 'converted_accession')
mergedSRA = mergedSRA.drop_duplicates()

# merge GEO attributes onto table of pmcData + SRA Attributes
allFactors = pd.merge(mergedSRA, slicedGEOAtt, how = 'left', on = 'converted_accession')
allFactors = pd.merge(allFactors, ls_guesses, how = 'left', left_on = 'geoHardware', right_on = 'hardware')

allFactors = allFactors.dropna(subset = ['converted_accession'])

In [29]:
# clean up columns with factor for both SRA and GEO, rearrange columns
allFactors['species'] = allFactors['sraSpecies'].fillna(allFactors['geoSpecies'])
allFactors = allFactors.drop(labels = ['sraSpecies', 'geoSpecies'], axis = 1)

allFactors['hardware'] = allFactors['sraHardware'].fillna(allFactors['geoHardware'])
allFactors = allFactors.drop(labels = ['sraHardware', 'geoHardware'], axis = 1)

allFactors['library_strategy'] = allFactors['sraLibrary_strategy'].fillna(allFactors['geoLibrary_strategy'])
allFactors = allFactors.drop(labels = ['sraLibrary_strategy', 'geoLibrary_strategy'], axis = 1)

allFactors['repository_date'] = allFactors['sraRelease'].fillna(allFactors['geoRelease'])
allFactors = allFactors.drop(labels = ['sraRelease', 'geoRelease'], axis = 1)

cols = ['journal', 'pmc_ID', 'accession', 'converted_accession', 'repository', 
        'pmc_date', 'repository_date', 'species', 
        'hardware', 'library_strategy', 'sraAvg_length', 'sraBases', 'sraAccess']

allFactors = allFactors[cols]
allFactors

,journal,pmc_ID,accession,converted_accession,repository,pmc_date,repository_date,species,hardware,library_strategy,sraAvg_length,sraBases,sraAccess
0,Biol_Direct,PMC4862193,SRR392998,GSE34743,GEO,2016-05-10,2012-03-15,Danio rerio,high-throughput sequencing,RNA-Seq,NaN,NaN,NaN
1,Biol_Direct,PMC4862193,SRR393000,GSE34743,GEO,2016-05-10,2012-03-15,Danio rerio,high-throughput sequencing,RNA-Seq,NaN,NaN,NaN
2,Biol_Direct,PMC4862193,SRR393006,GSE34743,GEO,2016-05-10,2012-03-15,Danio rerio,high-throughput sequencing,RNA-Seq,NaN,NaN,NaN
3,Biol_Direct,PMC4862193,SRR393008,GSE34743,GEO,2016-05-10,2012-03-15,Danio rerio,high-throughput sequencing,RNA-Seq,NaN,NaN,NaN
4,Biol_Direct,PMC4862193,SRR364364,GSE33671,GEO,2016-05-10,2011-12-09,Escherichia coli,high-throughput sequencing,RNA-Seq,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
223959,Saudi_J_Biol_Sci,PMC6088103,GSE28146,GSE28146,GEO,2018-05-18,2011-08-01,Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN
223960,Saudi_J_Biol_Sci,PMC6933160,ERS2923991,ERP109196,SRA,2019-05-07,2019-01-24,metagenome,454 GS FLX,AMPLICON,250.0,6461726.0,public
223961,Saudi_J_Biol_Sci,PMC6933160,ERS2923992,ERP109196,SRA,2019-05-07,2019-01-24,metagenome,454 GS FLX,AMPLICON,250.0,6461726.0,public
223962,Saudi_J_Biol_Sci,PMC6933160,ERS2923995,ERP109196,SRA,2019-05-07,2019-01-24,metagenome,454 GS FLX,AMPLICON,250.0,6461726.0,public


In [30]:
# save to .csv
allFactors.to_csv('../data_tables/metadataMatrix_raw.csv', index = False)